In [ ]:
#Chatbot

In [1]:
import numpy as np 
import tensorflow as tf
import pickle
from tensorflow.keras import layers, activations, models, preprocessing, utils

2024-03-08 11:15:15.142017: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
import pandas as pd
df = pd.read_csv('chatbot.csv')


In [5]:
questions = list(df['question'])
answers = list(df['answer'])
print(questions[:2])
print(answers[:2])

['have you read the communist', 'what is a government']
['yes, marx had made some interesting observations.', 'ideally it is a representative of the people.']


In [6]:
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( ['start']+questions + answers )
VOCAB_SIZE = len( tokenizer.word_index )+1
print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))

VOCAB SIZE : 1589


Preparing data for Seq2Seq model
This model requires 3 arrays encoder_input_data, decoder_input_data and decoder_output_data.

For encoder_input_data: Tokensize the Questions and Pad them to their maximum Length.

For decoder_input_data: Tokensize the Answers and Pad them to their maximum Length.

For decoder_output_data: Tokensize the Answers and Remove the 1st element from all the tokenized_answers. This is the element which was added earlier.

In [7]:
from gensim.models import Word2Vec
import re

vocab = []
for word in tokenizer.word_index:
    vocab.append(word)

def tokenize(sentences):
    tokens_list = []
    vocabulary = []
    for sentence in sentences:
        sentence = sentence.lower()
        sentence = re.sub('[^a-zA-Z]', ' ', sentence)
        tokens = sentence.split()
        vocabulary += tokens
        tokens_list.append(tokens)
        return tokens_list, vocabulary

In [8]:
#encoder_input_data
tokenized_questions = tokenizer.texts_to_sequences( questions )
maxlen_questions = max( [len(x) for x in tokenized_questions ] )
padded_questions = preprocessing.sequence.pad_sequences( tokenized_questions, maxlen = maxlen_questions, padding = 'post')
encoder_input_data = np.array(padded_questions)
print(encoder_input_data.shape, maxlen_questions)
print(encoder_input_data[0])
     

(566, 15) 15
[ 21   1  57   5 635   0   0   0   0   0   0   0   0   0   0]


In [73]:
# decoder_input_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
maxlen_answers = max( [ len(x) for x in tokenized_answers ] )
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
decoder_input_data = np.array( padded_answers )
print( decoder_input_data.shape , maxlen_answers )
print(decoder_input_data[0])

(566, 34) 34
[103 815 316 165  88 475 816   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]


In [74]:
# decoder_output_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
onehot_answers = utils.to_categorical( padded_answers , VOCAB_SIZE )
decoder_output_data = np.array( onehot_answers )
print( decoder_output_data.shape )

(566, 34, 1589)


# Step 4: Defining Encoder Decoder Model

In [80]:
encoder_inputs = tf.keras.layers.Input(shape=( maxlen_questions , ))
encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h , state_c = tf.keras.layers.LSTM( 200 , return_state=True )( encoder_embedding )
encoder_states = [ state_h , state_c ]

decoder_inputs = tf.keras.layers.Input(shape=( maxlen_answers ,  ))
decoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True) (decoder_inputs)
decoder_lstm = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm ( decoder_embedding , initial_state=encoder_states )
decoder_dense = tf.keras.layers.Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')

model.summary()

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_13 (InputLayer)          [(None, 15)]         0           []                               
                                                                                                  
 input_14 (InputLayer)          [(None, 34)]         0           []                               
                                                                                                  
 embedding_6 (Embedding)        (None, 15, 200)      317800      ['input_13[0][0]']               
                                                                                                  
 embedding_7 (Embedding)        (None, 34, 200)      317800      ['input_14[0][0]']               
                                                                                            

# Step 5: Training the Model

In [81]:
model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=50, epochs=100 ) 
#model.save( 'model.h5' )

Epoch 1/100
12/12 [==============================] - 8s 138ms/step - loss: 1.6885
Epoch 2/100
12/12 [==============================] - 2s 137ms/step - loss: 1.4573
Epoch 3/100
12/12 [==============================] - 2s 139ms/step - loss: 1.4271
Epoch 4/100
12/12 [==============================] - 2s 155ms/step - loss: 1.4130
Epoch 5/100
12/12 [==============================] - 2s 144ms/step - loss: 1.3929
Epoch 6/100
12/12 [==============================] - 2s 151ms/step - loss: 1.3759
Epoch 7/100
12/12 [==============================] - 2s 154ms/step - loss: 1.3612
Epoch 8/100
12/12 [==============================] - 2s 154ms/step - loss: 1.3400
Epoch 9/100
12/12 [==============================] - 2s 144ms/step - loss: 1.3210
Epoch 10/100
12/12 [==============================] - 2s 174ms/step - loss: 1.3017
Epoch 11/100
12/12 [==============================] - 2s 141ms/step - loss: 1.2819
Epoch 12/100
12/12 [==============================] - 2s 186ms/step - loss: 1.2617
Epoch 13/100


In [77]:
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    
    decoder_states = [state_h, state_c]

    decoder_outputs = decoder_dense(decoder_outputs)
    
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model
     

Talking with the Chatbot
define a method str_to_tokens which converts str questions to Integer tokens with padding.

First, we take a question as input and predict the state values using enc_model.
We set the state values in the decoder's LSTM.
Then, we generate a sequence which contains the element.
We input this sequence in the dec_model.
We replace the element with the element which was predicted by the dec_model and update the state values.
We carry out the above steps iteratively till we hit the tag or the maximum answer length.

In [78]:
def str_to_tokens( sentence : str ):

    words = sentence.lower().split()
    tokens_list = list()
  
    for word in words:
        tokens_list.append( tokenizer.word_index[ word ] ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen_questions , padding='post')

In [82]:
enc_model , dec_model = make_inference_models()

for _ in range(10):
    states_values = enc_model.predict( str_to_tokens( input( 'Enter question : ' ) ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( decoded_translation )

Enter question : what is your name
1/1 [==============================] - 0s 21ms/step
 is a chat robot business same you same same i can same as i can same as a resemblance as being same as being same as best same as i can be same as a
Enter question : what is a government
1/1 [==============================] - 0s 21ms/step
 is a hypothetical question same same i think the conversation or the study of the movies same same i can be same as gossip same same as i can same as a resemblance between us


KeyboardInterrupt: Interrupted by user